In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import *
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor
import random

In [8]:
train= pd.read_csv("E:/TookiTaki/My work/Kaggle/Restaurant Revenue Prediction/train.csv/Revenue Prediction train.csv")
test= pd.read_csv("E:/TookiTaki/My work/Kaggle/Restaurant Revenue Prediction/test.csv/test.csv")

In [9]:
train.revenue.describe()

count    1.370000e+02
mean     4.453533e+06
std      2.576072e+06
min      1.149870e+06
25%      2.999068e+06
50%      3.939804e+06
75%      5.166635e+06
max      1.969694e+07
Name: revenue, dtype: float64

In [10]:
test.head()

,Id,Open Date,City,City Group,Type,P1,P2,P3,P4,P5,...,P28,P29,P30,P31,P32,P33,P34,P35,P36,P37
0,0,01/22/2011,Niğde,Other,FC,1,4.0,4.0,4.0,1,...,2.0,3.0,0,0,0,0,0,0,0,0
1,1,03/18/2011,Konya,Other,IL,3,4.0,4.0,4.0,2,...,1.0,3.0,0,0,0,0,0,0,0,0
2,2,10/30/2013,Ankara,Big Cities,FC,3,4.0,4.0,4.0,2,...,2.0,3.0,0,0,0,0,0,0,0,0
3,3,05/06/2013,Kocaeli,Other,IL,2,4.0,4.0,4.0,2,...,2.0,3.0,0,4,0,0,0,0,0,0
4,4,07/31/2013,Afyonkarahisar,Other,FC,2,4.0,4.0,4.0,1,...,5.0,3.0,0,0,0,0,0,0,0,0


In [13]:
train.rename(columns={'Open Date': 'Open_Date'},inplace= True )
test.rename(columns={'Open Date': 'Open_Date'},inplace= True)

In [14]:
train.Open_Date=pd.to_datetime(train.Open_Date)
test.Open_Date=pd.to_datetime(test.Open_Date)

In [15]:
pd.to_datetime('20150101', format='%Y%m%d')

Timestamp('2015-01-01 00:00:00')

In [16]:
train['Open_Days']=pd.to_datetime('20150101', format='%Y%m%d')-train.Open_Date
train['Open_Days']=(train.Open_Days/np.timedelta64(1,'D')).astype(int)

In [17]:
test['Open_Days']=pd.to_datetime('20150101', format='%Y%m%d')-test.Open_Date
test['Open_Days']=(test.Open_Days/np.timedelta64(1,'D')).astype(int)

In [18]:
train['Open_month']=train.Open_Date.dt.month
train['Open_year']= train.Open_Date.dt.year
#train['revenue']= np.sqrt(np.log(train['revenue']))
train.head()

,Id,Open_Date,City,City Group,Type,P1,P2,P3,P4,P5,...,P32,P33,P34,P35,P36,P37,revenue,Open_Days,Open_month,Open_year
0,0,1999-07-17,İstanbul,Big Cities,IL,4,5.0,4.0,4.0,2,...,4,5,5,4,3,4,5653753.0,5647,7,1999
1,1,2008-02-14,Ankara,Big Cities,FC,4,5.0,4.0,4.0,1,...,0,0,0,0,0,0,6923131.0,2513,2,2008
2,2,2013-03-09,Diyarbakır,Other,IL,2,4.0,2.0,5.0,2,...,0,0,0,0,0,0,2055379.0,663,3,2013
3,3,2012-02-02,Tokat,Other,IL,6,4.5,6.0,6.0,4,...,10,6,18,12,12,6,2675511.0,1064,2,2012
4,4,2009-05-09,Gaziantep,Other,IL,3,4.0,3.0,4.0,2,...,3,2,3,4,3,3,4316715.0,2063,5,2009


In [15]:
# from sklearn import preprocessing
# le_Type = preprocessing.LabelEncoder()
# train.Type= le_Type.fit_transform(train.Type)
# train.City=le_Type.fit_transform(train.City)
# train.head()


In [19]:
test['Open_month']=test.Open_Date.dt.month
test['Open_year']=test.Open_Date.dt.year
test['Type']=test['Type'].replace(['MB'], 'DT')
test.head()

,Id,Open_Date,City,City Group,Type,P1,P2,P3,P4,P5,...,P31,P32,P33,P34,P35,P36,P37,Open_Days,Open_month,Open_year
0,0,2011-01-22,Niğde,Other,FC,1,4.0,4.0,4.0,1,...,0,0,0,0,0,0,0,1440,1,2011
1,1,2011-03-18,Konya,Other,IL,3,4.0,4.0,4.0,2,...,0,0,0,0,0,0,0,1385,3,2011
2,2,2013-10-30,Ankara,Big Cities,FC,3,4.0,4.0,4.0,2,...,0,0,0,0,0,0,0,428,10,2013
3,3,2013-05-06,Kocaeli,Other,IL,2,4.0,4.0,4.0,2,...,4,0,0,0,0,0,0,605,5,2013
4,4,2013-07-31,Afyonkarahisar,Other,FC,2,4.0,4.0,4.0,1,...,0,0,0,0,0,0,0,519,7,2013


In [17]:
#from sklearn import preprocessing
#le_Type = preprocessing.LabelEncoder()
#test.Type= le_Type.fit_transform(test.Type)
#test.City=le_Type.fit_transform(test.City)
#test.head()


In [20]:
train['Difference']='Train' 
test['Difference']='Test'
fullData = pd.concat([train,test],axis=0)
fullData.Difference.describe()

count     100137
unique         2
top         Test
freq      100000
Name: Difference, dtype: object

In [21]:
from sklearn import preprocessing
le_Type = preprocessing.LabelEncoder()
fullData.Type= le_Type.fit_transform(fullData.Type)
fullData.City=le_Type.fit_transform(fullData.City)
fullData.head()

,City,City Group,Difference,Id,Open_Date,Open_Days,Open_month,Open_year,P1,P10,...,P36,P37,P4,P5,P6,P7,P8,P9,Type,revenue
0,60,Big Cities,Train,0,1999-07-17,5647,7,1999,4,5,...,3,4,4.0,2,2,5,4,5,2,5653753.0
1,4,Big Cities,Train,1,2008-02-14,2513,2,2008,4,5,...,0,0,4.0,1,2,5,5,5,1,6923131.0
2,14,Other,Train,2,2013-03-09,663,3,2013,2,5,...,0,0,5.0,2,3,5,5,5,2,2055379.0
3,52,Other,Train,3,2012-02-02,1064,2,2012,6,10,...,12,6,6.0,4,4,10,8,10,2,2675511.0
4,21,Other,Train,4,2009-05-09,2063,5,2009,3,5,...,3,3,4.0,2,2,5,5,5,2,4316715.0


In [22]:
train=fullData[fullData['Difference']=='Train']
test=fullData[fullData['Difference']=='Test']
train.revenue.describe()
train.head()

,City,City Group,Difference,Id,Open_Date,Open_Days,Open_month,Open_year,P1,P10,...,P36,P37,P4,P5,P6,P7,P8,P9,Type,revenue
0,60,Big Cities,Train,0,1999-07-17,5647,7,1999,4,5,...,3,4,4.0,2,2,5,4,5,2,5653753.0
1,4,Big Cities,Train,1,2008-02-14,2513,2,2008,4,5,...,0,0,4.0,1,2,5,5,5,1,6923131.0
2,14,Other,Train,2,2013-03-09,663,3,2013,2,5,...,0,0,5.0,2,3,5,5,5,2,2055379.0
3,52,Other,Train,3,2012-02-02,1064,2,2012,6,10,...,12,6,6.0,4,4,10,8,10,2,2675511.0
4,21,Other,Train,4,2009-05-09,2063,5,2009,3,5,...,3,3,4.0,2,2,5,5,5,2,4316715.0


In [24]:
ID_col=['Id']
target_col=['revenue']
cat_col=['City Group','Open_Date','Difference']
num_cols=list(set(list(fullData.columns))-set(ID_col)-set(target_col)-set(cat_col))

In [25]:
Id_dummy=test.Id
Id_new=train.Id
features=list(set(list(fullData.columns))-set(ID_col)-set(target_col)-set(cat_col))
# x_train = train[list(features)].values
# y_train = train["revenue"].values
# x_test=test[list(features)].values

In [61]:
#test[features].isnull().sum()

In [27]:
#model = svm.SVC(kernel='rbf', C=1, gamma=1) 
#model.fit(x_train, y_train)
#model.score(x_train, y_train)
cv=[]
lst=[]
kf=cross_validation.KFold(len(train),n_folds=5,random_state=0)
for idx1,idx2 in kf:
    x_train,x_cv=train[features].iloc[idx1],train[features].iloc[idx2]
    y_train,y_cv=train.revenue.iloc[idx1],train.revenue.iloc[idx2]
    
    random.seed(100)
    rf = RandomForestRegressor(n_estimators=500,max_depth=20,max_leaf_nodes=2)
    rf.fit(x_train, y_train)
    cv.extend(rf.predict(x_cv))
    lst.append(rf.predict(test[features]))
    print(metrics.mean_absolute_error(rf.predict(x_cv),y_cv))

1969435.42893
1506382.62338
1818354.14328
1594856.83015
1766969.9533


In [2]:
importances = rf.feature_importances_
std = np.std([tree.feature_importances_ for tree in rf.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]
print("Feature ranking:")
for f in range(x_train.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

NameError: name 'rf' is not defined

In [21]:
df1=pd.DataFrame({'Id': Id_dummy, 'revenue':np.average(lst,axis=0)})
df2=pd.DataFrame({'Id': Id_new,'revenue': cv})

In [37]:
#df2.head()

In [38]:
#train.head()

In [39]:
#df3.head()

In [79]:
#print(df1)


In [78]:
#print(df2)

In [22]:
#df1.revenue=np.exp(np.power(df1.revenue,2))
#print(df1.revenue)

In [40]:
#df2.revenue=np.exp(np.power(df2.revenue,2))
#print(df2.revenue)

In [41]:
df1.to_csv("C:/Users/Rajat/Desktop/RandomForest.csv",index= False)

In [24]:
df2.to_csv("C:/Users/Rajat/Desktop/RandomForestCV.csv",index= False)

In [56]:
x_train.shape[1]

42

In [73]:
importances = rf.feature_importances_
std = np.std([tree.feature_importances_ for tree in rf.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]
print("Feature ranking:")
for f in range(x_train.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

# Plot the feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(x_train.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(5), [['Open_Days'],['P30'],['P29'],['P16'],['P23']])
plt.xlim([-1, 5])
plt.show()

Feature ranking:
1. feature 5 (0.710000)
2. feature 29 (0.078000)
3. feature 28 (0.074000)
4. feature 15 (0.056000)
5. feature 22 (0.018000)
6. feature 4 (0.016000)
7. feature 10 (0.014000)
8. feature 20 (0.010000)
9. feature 14 (0.008000)
10. feature 11 (0.006000)
11. feature 24 (0.006000)
12. feature 27 (0.004000)
13. feature 7 (0.000000)
14. feature 8 (0.000000)
15. feature 9 (0.000000)
16. feature 17 (0.000000)
17. feature 6 (0.000000)
18. feature 3 (0.000000)
19. feature 12 (0.000000)
20. feature 13 (0.000000)
21. feature 2 (0.000000)
22. feature 1 (0.000000)
23. feature 16 (0.000000)
24. feature 41 (0.000000)
25. feature 18 (0.000000)
26. feature 19 (0.000000)
27. feature 39 (0.000000)
28. feature 38 (0.000000)
29. feature 37 (0.000000)
30. feature 36 (0.000000)
31. feature 35 (0.000000)
32. feature 34 (0.000000)
33. feature 33 (0.000000)
34. feature 32 (0.000000)
35. feature 31 (0.000000)
36. feature 30 (0.000000)
37. feature 26 (0.000000)
38. feature 25 (0.000000)
39. feature 2